## Imports

In [39]:
# To embed plots in the notebooks
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np # numpy library
import scipy . linalg as lng # linear algebra from scipy library
from scipy . spatial import distance # load distance function
from sklearn import preprocessing as preproc # load preprocessing function

# seaborn can be used to "prettify" default matplotlib plots by importing and setting as default
import seaborn as sns
sns.set() # Set searborn as default

## Load dataset

In [40]:
diabetPath = "./DiabetesDataNormalized.txt"
T = np.loadtxt(diabetPath, delimiter=" ", skiprows=1)
y = T[:, 10]
X = T[:, :10]

# Get number of observations (n) and number of independent variables (p)
[n, p] = np.shape(X)

M = X

## 1 Solve the Ordinary Least Squares (OLS) computationally (for the diabetes data set):

> (a) What is the difference between using a brute force implementation (analytical) for an OLS solver and a numerically ’smarter’ implementation? Compute the ordinary least squares solution to the diabetes data set for both options and look at the relative difference. Use for example lng.lstsq to invert the matrix or to solve the linear system of equation.

In [41]:
def ols_numerical(X, y):
    # Call lstsq from lng to get betas
    beta, _, _, _ = lng.lstsq(X, y)
    return beta


def ols_analytical(X, y):
    # Implement the analytical closed form way of calculating the betas
    X_transpose = np.transpose(X)
    beta = np.linalg.inv(X_transpose @ X) @ X_transpose @ y
    return beta

In [42]:
# numerical solution
beta_num = ols_numerical(M, y)
print(f"The list of betas: \n{beta_num}")

The list of betas: 
[-0.00618293 -0.14813008  0.32110005  0.20036692 -0.48931352  0.29447365
  0.06241272  0.10936897  0.46404908  0.04177187]


In [43]:
# analytical solution
beta_ana = ols_analytical(M, y)
print(f"The list of betas: \n{beta_ana}")

The list of betas: 
[-0.00618293 -0.14813008  0.32110005  0.20036692 -0.48931352  0.29447365
  0.06241272  0.10936897  0.46404908  0.04177187]


In [44]:
# difference in solutions
norm = np.linalg.norm(beta_ana - beta_num)
print(f"The norm of the difference between betas: \n{norm}")

The norm of the difference between betas: 
2.2328026633969272e-14


Is the difference significant?

What can we conclude relating to numerical vs analytical solutions?

The difference between the numerical and analytical solutions is negligible. This indicates that both methods are reliable for solving the Ordinary Least Squares (OLS) problem. However, the numerical method using `lstsq` is generally preferred for larger datasets because it is more stable and less prone to numerical errors compared to the analytical method, which involves matrix inversion.

> (b) How could you include an intercept term in Python? This means using the model: $y = β_0 +xβ_1 +...+x_pβ_p +ε $ rather than: $ y=x_1β_1 +...+x_pβ_p +ε. $

In [53]:
# Include offset / intercept adding a column of ones to the matrix
M = np.column_stack((np.ones(n), X))
# Recalculate beta_ana after adding the intercept term
beta_ana = ols_analytical(M, y)
beta_ana

array([-6.39787702e-16, -6.18292545e-03, -1.48130075e-01,  3.21100050e-01,
        2.00366920e-01, -4.89313521e-01,  2.94473646e-01,  6.24127211e-02,
        1.09368973e-01,  4.64049083e-01,  4.17718663e-02])

What is the value of the intercept coefficient?

Can you explain why?

The value of the intercept is -6.39787702e-16. The intercept is very close to zero due to the nature of the data preprocessing (centering and scaling) and the precision of the numerical computations.

> (c) Calculate the mean squared error $MSE = 1/n \sum^n_{i=1} (y_i−x_iβ)^2$.

In [46]:
# Calculate the estimated y values and use these to calculate the MSE.
def compute_mse(X, beta):
    y_est = X @ beta
    mse = np.mean((y - y_est) ** 2)
    return mse, y, y_est

In [47]:
mse_ana, res_ana, yhat_ana = compute_mse(M, beta_ana)
print(f"mse from the analytical solution: {mse_ana}")

mse from the analytical solution: 0.48116051086159695


What happens to the MSE if we change some of the betas?

Is that what you expected?

When we change some of the betas, the MSE (Mean Squared Error) increases. This is expected because altering the betas from their optimal values (as calculated by the OLS method) leads to a less accurate model, resulting in higher prediction errors and thus a higher MSE.

In [48]:
beta_new = beta_ana
beta_new[5] = 0

mse_new, res_new, yhat_new = compute_mse(M, beta_new)

print(f"mse from the changed betas: {mse_new}")

mse from the changed betas: 0.7200465405399794


> (d) Calculate the residual sum of squares $RSS = ||{\bf y} - X \beta||_2^2$ and the total sum of squares $TSS = ||{\bf y} - \bar{y}||_2^2$, where $\bar{y}$ is the estimated mean of ${\bf y}$. Report on the $R^2$ measure, that is, the proportion of variance in the sample set explained by the model: $R^2 = 1 - \frac{RSS}{TSS}$

In [55]:
rss = np.sum((y - M @ beta_ana) ** 2)
tss = np.sum((y - np.mean(y)) ** 2)
r2 = 1 - rss / tss
r2

np.float64(0.5177484222203499)

How much variance in <strong>y</strong> can we explain using this simple model?

The R-squared value for our model is 0.51775. This indicates that approximately 51.77% of the variance in y can be explained by this simple model.